## **Gradient Boosting in Regards to Heart Disease Classification, plus SHAP Visualisation for explaining the Models Nature**
*Group 10* <br>
*08/12/2025*

### **Data Preparation and Import**

Required external pip imports will be retrieved and downloaded in the below pip install code cell for the notebook requirments to be resolved.

In [ ]:
%pip install pandas pd numpy scikit-learn matplotlib seaborn kagglehub shap xgboost lightgbm dask dask-ml distributed ipywidgets 

Now the notebook will import the downloaded pip modules ensuring they can be linked, retrieved and used globally in the notebook.

In [ ]:
import numpy as np # for performing more advanced operations on arrays, such as converting the heart disease dataset columns to float64 and int64 respectively as well as creating partions from the x training set
import pandas as pd # for converting the heart disease dataset to a more robustly interfacable pandas dataframe array for working with the various ML Models and helper methods
import matplotlib.pyplot as plt # plotting roc scores et al of the gradient boosting models
import seaborn as sns # for heatmap plotting the correlation matrix of the models

import kagglehub # for retrieving and downloading the heart disease dataset from kaggle

# Gradient Boosting Models
import xgboost as xgb # the xgboost model
import lightgbm as lgb # the lightgbm model

import time # for measuring inference time of the models
from math import pi # for computing pie angles in evaluation plots

# SHAP 
import shap # explaining how the two gradient boosting models made their predictions, i.e. feature importance in regards to the two grad boost models created deeper within this notebook

# Scikit Learn Methods
from sklearn.preprocessing import StandardScaler # for scaling the heart disease dataset features
from sklearn.model_selection import train_test_split # for splitting the heart disease dataset into training, testing and validation sets
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score  # for getting the scores, precision, recall, accuracy and f1 scores for evaulating the different metric scores for each model
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score # for generating model classification reports, confusion matrices and roc auc scores
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV # for selecting best hyperparameters for the gradient boosting models, both using 10 fold cross validation

# Dask for distributed learning of the LightGBM model
from dask.distributed import Client # contructing the dask client for distributed learning
from dask_ml.model_selection import GridSearchCV as DaskGridSearchCV # selecting best hyperparameters for the LightGBM model using dask distributed learning
import dask.array as da # data collection for the dask distributed learning client

import warnings # hiding warnings to clean up evaluation output

warnings.filterwarnings('ignore') # filter out any warnings in the notebook cell outputs

### **Data Exploration**

In [ ]:
path = kagglehub.dataset_download("redwankarimsony/heart-disease-data") # get path to downloaded kaggle heart disease dataset files

print("Path to kaggle files:", path) # output to user that path

In [ ]:
heart_data = path + "/heart_disease_uci.csv" # retrieve the dataset by appending the filename to the enclosing path

In [ ]:
heart_disease_df = pd.read_csv(heart_data) #convert heart disease csv data to a pandas dataframe

In [ ]:
print("Heart Disease Dataset before Preprocessing:")
heart_disease_df.head() # head the dataset to see that it has been converted to a pandas dataframe successfully

In [ ]:
heart_disease_df.shape # get rows and columns size of the heart disease dataset 

In [ ]:
# Binarisation of the Target 'Num' Feature
heart_disease_df["num"] = heart_disease_df["num"].apply(lambda x: 1 if x > 0 else 0) # make target num feature binary, either true 1 or 0 

In [ ]:
print("Heart Disease Dataset after Target Binarisation:")
heart_disease_df.head() # show newly binarised target feature has been applied successfully

In [ ]:
heart_disease_df.info() # output number of columns, along with their names, data type and how many non-null values they contain

### **Data Pre-Processing**

In [ ]:
heart_disease_df = heart_disease_df.drop(columns=["id"]) # drop index id column as it does not need to be represented 

print("Heart Disease Dataset without ID Column:")
heart_disease_df.head() # show dataset with the dropped id column, notying we still have an index

In [ ]:
categorical_cols = heart_disease_df.select_dtypes(include=['object']).columns.tolist() # find categorical columns in the dataset by filtering for object data types

print("Categorical Columns in Heart Disease Dataset:")
categorical_cols # output textually the categorical columns found above

In [ ]:
heart_disease_df = pd.get_dummies(heart_disease_df, columns=categorical_cols, drop_first=True) # one hot encode the categorical columns found above

In [ ]:
print("Missing Values in Each Feature:")
heart_disease_df.isnull().sum() # search for missing values in the heart disease dataset

In [ ]:
missing_threshold = 0.5 # if over half the values in a column are missing, drop the column

for col in heart_disease_df.columns: # iterate through the heart disease  columns
    missing_fraction = heart_disease_df[col].isnull().mean() # compute the fraction of missing values in the column

    if missing_fraction > missing_threshold: # if the missing fraction is greater than the threshold
        heart_disease_df.drop(columns=[col], inplace=True) # drop the column

    else: # if the missing fraction is less than the threshold
        if heart_disease_df[col].dtype in [np.float64, np.int64]: # and if the column is numerical
            heart_disease_df[col] = heart_disease_df[col].fillna(heart_disease_df[col].median()) # fill missing values with the median value of the column

        else: # if the column is categorical
            heart_disease_df[col] = heart_disease_df[col].fillna(heart_disease_df[col].mode()[0]) # fill missing values with the mode value of the column

heart_disease_df.isnull().sum() # now verify that there are no missing values in the heart disease dataset

heart_disease_df.head() # output the cleaned heart disease dataset with no remaining missing values

In [ ]:
numerical_cols = heart_disease_df.select_dtypes(include=[np.float64, np.int64]).columns.tolist() # get numerical columns by filtering for float64 and int64 data types

numerical_cols.remove("num") # remove the target variable column from the list of numerical columns

scaler = StandardScaler() # apply standard scaling to the numerical columns to bring them to a common scale

heart_disease_df[numerical_cols] = scaler.fit_transform(heart_disease_df[numerical_cols]) # fit and transform the numerical columns using the standard scaler

heart_disease_df.head() # output the heart disease dataset with scaled numerical features

In [ ]:
plt.figure(figsize=(12, 10)) # setup a plot

correlation_matrix = heart_disease_df.corr() # get correlation matrix of the heart disease dataset

sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True) # heatmap plot the correlation matrix

plt.title('Correlation Heatmap of Heart Disease Features', fontsize=16) 

plt.show() # output heatmap plot of the correlation matrix

In [ ]:
X = heart_disease_df.drop("num", axis=1) # get the features of heart disease by dropping the target variable column

y = heart_disease_df["num"] # get the target variable column of heart disease

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=65) # split the heart disease dataset into training and testing sets with an 80-20 split

## **Introducing the Gradient Boosting Models**

### **Training the XGBoost Model**

In [ ]:
xgb_clf = xgb.XGBClassifier(eval_metric='logloss', random_state=65) # setup the xgboost classifier model

xgb_clf.fit(X_train, y_train) # train the xgboost classifier model on the training data of the heart disease dataset

In [ ]:
start_time = time.time()
y_pred_xgb = xgb_clf.predict(X_test) # make predictions
inference_time_xgb = time.time() - start_time

print(confusion_matrix(y_test, y_pred_xgb)) # output confusion matrix of the xgboost model

print(classification_report(y_test, y_pred_xgb)) # output classification report of the xgboost model

print('ROC AUC:', roc_auc_score(y_test, y_pred_xgb)) # output ROC AUC score of the xgboost model

### **Finding Best Hyperparameters for XGBoost**

In [ ]:
# XGBoost hyperparameters for grid search
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'min_child_weight': [1, 3, 5]
}

In [ ]:
# setting up xgboost to search the grid in the above cell to find best hyperparameters
xgb_grid = GridSearchCV(xgb.XGBClassifier(random_state=65),
                      xgb_param_grid, # use the custom param grid
                      cv=10, # 10 cross folds validations
                      scoring='roc_auc', # score based on ROC AUC
                      n_jobs=-1, # use every cpu core available
                      verbose=1) # setup the xgboost grid search with 10 fold cross validation to find best hyperparameters from the parameter grid above

xgb_grid.fit(X_train, y_train) # perform the grid search on the training data

print('Best XGBoost Params:', xgb_grid.best_params_) # report back the best hyperparameters found from the grid search
print('Best XGBoost CV ROC AUC:', xgb_grid.best_score_) # report back the best cross-validated ROC AUC score from the grid search

## **Training a Non-Distributed LightGBM**

In [ ]:
lgb_clf = lgb.LGBMClassifier(random_state=65) # setup the lightgbm classifier model

lgb_clf.fit(X_train, y_train) # train the lightgbm classifier model on the training data of the heart disease dataset

In [ ]:
y_pred_lgb = lgb_clf.predict(X_test) # make predictions

print(confusion_matrix(y_test, y_pred_lgb)) # output confusion matrix of the lightgbm model all using the testing data split

print(classification_report(y_test, y_pred_lgb)) # output the classification report of the lightgbm model

print('ROC AUC:', roc_auc_score(y_test, y_pred_lgb)) # output ROC AUC score of the lightgbm model

### **Finding Best Hyperparameters for the Non-Distributed LightGBM**

In [ ]:
# LightGBM hyperparameter grid for the randomised lgb search using cross validation
lgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [-1, 5, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.8, 0.9, 1.0],
    'min_child_samples': [5, 10, 20]
}

In [ ]:
# setup the randomised search from the hyperparameters defined in the grid above
lgb_grid = RandomizedSearchCV(
    lgb.LGBMClassifier(random_state=65), # instance the lgbm classifier
    lgb_param_grid, # use the cusotm hyperparameter grid
    n_iter=20, # 20 random search iterations
    cv=10, # use 10 fold cross validation
    scoring='roc_auc', # score based on ROC AUC
    n_jobs=-1, # use every cpu core available
    verbose=1, # output more detailed lgb output
    random_state=65
) # use randomized search cv to find best hyperparameters for lightgbm model for 10 fold cross validation

lgb_grid.fit(X_train, y_train) # perform the randomized search on the training data

print('Best LightGBM Params:', lgb_grid.best_params_)  # output the best hyperparameters found for lightgbm model
print('Best LightGBM CV ROC AUC:', lgb_grid.best_score_) # output the best cross-validated ROC AUC score from the randomized search

## **Training a Distributed LightGBM**

In [ ]:
# LightGBM with distributed learning using Dask

# setup a dask client for applying distributed learning to the lightgbm model, with 2 workers, hyperthreading and 2GB memory limit per worker
client = Client(n_workers=2, threads_per_worker=2, memory_limit='2GB')

print(client) # output parameters of the dask client

print(f"Dashboard link: {client.dashboard_link}") # output the dashboard link for the dask client

In [ ]:
# convert training and testing data to dask arrays with appropriate chunk sizes for distributed learning using dask
X_train_dask = da.from_array(X_train.values, chunks=(len(X_train)//2, X_train.shape[1])) # convert X training set to dask array with 2 partitions
y_train_dask = da.from_array(y_train.values, chunks=len(y_train)//2) # convert y training set to dask array with 2 partitions
X_test_dask = da.from_array(X_test.values, chunks=(len(X_test)//2, X_test.shape[1])) # convert X testing set to dask array with 2 partitions

print(f"Training data partitions: {X_train_dask.npartitions}") # output number of partitions in the dask array
print(f"Training data shape: {X_train_dask.shape}") # output shape of the dask array

In [ ]:
# Train LightGBM with distributed learning
lgb_dist_clf = lgb.LGBMClassifier(
    random_state=65,
    n_jobs=-1  # utilize all available CPU cores
)

lgb_dist_clf.fit(X_train, y_train) # fit on the heart disease training data

y_pred_lgb_dist = lgb_dist_clf.predict(X_test) # make predictions using distrubted LightGBM model

print("LightGBM Distributed Learning Results:") # header for results

print(confusion_matrix(y_test, y_pred_lgb_dist))  # output confusion matrix of the distributed lightgbm model

print(classification_report(y_test, y_pred_lgb_dist)) # output classification report of the distributed lightgbm model

print('ROC AUC:', roc_auc_score(y_test, y_pred_lgb_dist)) # output ROC AUC score of the distributed lightgbm model

### **Finding Best Hyperparameters for the Distributed LightGBM**

In [ ]:
lgb_dist_grid = DaskGridSearchCV(
    lgb.LGBMClassifier(random_state=65, n_jobs=1), # instance the lgbm classifier again
    lgb_param_grid, # use the same custom lgb hyperparameter grid
    cv=10, # use 10 cross folds for cross validation
    scoring='roc_auc' # score based on ROC AUC
) # setup dask grid search cv for distributed learning to find best hyperparameters for lightgbm model

lgb_dist_grid.fit(X_train, y_train) # perform the grid search on the training data

print('Best LightGBM Distributed Params:', lgb_dist_grid.best_params_) # output the best hyperparameters found for distributed lightgbm model

print('Best LightGBM Distributed CV ROC AUC:', lgb_dist_grid.best_score_) # output the best cross-validated ROC AUC score from the distributed grid search

In [ ]:
client.close() # close the dask client as we have found best hyperparameters and distributed learning is done

## **Evaluation of Gradient Boosting Model Performance**

In [ ]:
# XGBoost evaluation
results = {}

precision_xgb = precision_score(y_test, y_pred_xgb)
recall_xgb = recall_score(y_test, y_pred_xgb)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, y_pred_xgb)

results['XGBoost'] = {
    'Precision': precision_xgb,
    'Recall': recall_xgb,
    'Accuracy': accuracy_xgb,
    'F1-Score': f1_xgb,
    'ROC AUC': roc_auc_xgb,
    'Inference Time (s)': inference_time_xgb,
}

print("XGBoost Evaluation")
print("=" * 60)
print(f"Precision:  {precision_xgb:.4f}")
print(f"Recall:     {recall_xgb:.4f}")
print(f"Accuracy:   {accuracy_xgb:.4f}")
print(f"F1-Score:   {f1_xgb:.4f}")
print(f"ROC AUC:    {roc_auc_xgb:.4f}")
print(f"Inference Time: {inference_time_xgb:.4f}s")

In [ ]:
# LightGBM (non-distributed) evaluation
print("\n" + "=" * 60)
print("LightGBM (Non-Distributed) Model Evaluation")
print("=" * 60)

start_time = time.time()
y_pred_lgb = lgb_clf.predict(X_test)
inference_time_lgb = time.time() - start_time

precision_lgb = precision_score(y_test, y_pred_lgb)
recall_lgb = recall_score(y_test, y_pred_lgb)
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
f1_lgb = f1_score(y_test, y_pred_lgb)
roc_auc_lgb = roc_auc_score(y_test, y_pred_lgb)

results['LightGBM'] = {
    'Precision': precision_lgb,
    'Recall': recall_lgb,
    'Accuracy': accuracy_lgb,
    'F1-Score': f1_lgb,
    'ROC AUC': roc_auc_lgb,
    'Inference Time (s)': inference_time_lgb,
}

print(f"Precision:  {precision_lgb:.4f}")
print(f"Recall:     {recall_lgb:.4f}")
print(f"Accuracy:   {accuracy_lgb:.4f}")
print(f"F1-Score:   {f1_lgb:.4f}")
print(f"ROC AUC:    {roc_auc_lgb:.4f}")
print(f"Inference Time: {inference_time_lgb:.4f}s")

In [ ]:
# LightGBM (distributed) evaluation
print("\n" + "=" * 60)
print("LightGBM (Distributed) Model Evaluation")
print("=" * 60)

start_time = time.time()
y_pred_lgb_dist = lgb_dist_clf.predict(X_test)
inference_time_lgb_dist = time.time() - start_time

precision_lgb_dist = precision_score(y_test, y_pred_lgb_dist)
recall_lgb_dist = recall_score(y_test, y_pred_lgb_dist)
accuracy_lgb_dist = accuracy_score(y_test, y_pred_lgb_dist)
f1_lgb_dist = f1_score(y_test, y_pred_lgb_dist)
roc_auc_lgb_dist = roc_auc_score(y_test, y_pred_lgb_dist)

results['LightGBM Distributed'] = {
    'Precision': precision_lgb_dist,
    'Recall': recall_lgb_dist,
    'Accuracy': accuracy_lgb_dist,
    'F1-Score': f1_lgb_dist,
    'ROC AUC': roc_auc_lgb_dist,
    'Inference Time (s)': inference_time_lgb_dist,
}

print(f"Precision:  {precision_lgb_dist:.4f}")
print(f"Recall:     {recall_lgb_dist:.4f}")
print(f"Accuracy:   {accuracy_lgb_dist:.4f}")
print(f"F1-Score:   {f1_lgb_dist:.4f}")
print(f"ROC AUC:    {roc_auc_lgb_dist:.4f}")
print(f"Inference Time: {inference_time_lgb_dist:.4f}s")

In [ ]:
# Results table
print("\n" + "=" * 60)
print("Comprehensive Model Comparison Table")
print("=" * 60)

results_df = pd.DataFrame(results).T
metrics = ['Precision', 'Recall', 'Accuracy', 'F1-Score', 'ROC AUC']

print(results_df.to_string())

In [ ]:
# Metrics bar chart + inference time
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Model Performance Metrics Comparison', fontsize=16, fontweight='bold')

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, metric in enumerate(metrics):
    ax = axes[idx // 3, idx % 3]
    values = [results[model][metric] for model in results.keys()]
    bars = ax.bar(results.keys(), values, color=colors)

    ax.set_ylabel(metric, fontweight='bold')
    ax.set_ylim([0, 1.0])
    ax.set_title(metric)

    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2.0, height, f'{height:.3f}',
        ha='center', va='bottom', fontsize=9)

ax = axes[1, 2]
inference_times = [results[model]['Inference Time (s)'] for model in results.keys()]
bars = ax.bar(results.keys(), inference_times, color=colors)

ax.set_ylabel('Time (seconds)', fontweight='bold')
ax.set_title('Inference Time')

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2.0, height, f'{height:.4f}s',
    ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Heatmap of all metrics
fig, ax = plt.subplots(figsize=(10, 5))
sns.heatmap(results_df, annot=True, fmt='.4f', cmap='RdYlGn', cbar_kws={'label': 'Score'}, ax=ax)

ax.set_title('Model Performance Heatmap', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Radar chart comparison
categories = ['Precision', 'Recall', 'Accuracy', 'F1-Score', 'ROC AUC']
angles = [n / float(len(categories)) * 2 * pi for n in range(len(categories))]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

colors_radar = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, model in enumerate(results.keys()):
    values = [results[model][cat] for cat in categories]
    values += values[:1]
    
    ax.plot(angles, values, 'o-', linewidth=2, label=model, color=colors_radar[idx])
    ax.fill(angles, values, alpha=0.15, color=colors_radar[idx])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories)
ax.set_ylim(0, 1)
ax.set_title('Model Performance Radar Chart', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Individual metrics line plot
fig, ax = plt.subplots(figsize=(12, 6))
x_pos = np.arange(len(metrics))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, model in enumerate(results.keys()):
    values = [results[model][metric] for metric in metrics]
    
    ax.plot(x_pos, values, marker='o', label=model, linewidth=2, markersize=8, color=colors[idx])

ax.set_xlabel('Metrics', fontweight='bold', fontsize=12)
ax.set_ylabel('Score', fontweight='bold', fontsize=12)
ax.set_title('Model Performance Trend Across Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(metrics, rotation=15)
ax.set_ylim([0, 1.05])
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Grouped bar chart for key metrics
fig, ax = plt.subplots(figsize=(12, 6))
key_metrics = ['Precision', 'Recall', 'F1-Score']
x_pos = np.arange(len(key_metrics))
width = 0.25
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

for idx, model in enumerate(results.keys()):
    values = [results[model][metric] for metric in key_metrics]
    
    ax.bar(x_pos + idx * width, values, width, label=model, color=colors[idx])

ax.set_xlabel('Metrics', fontweight='bold', fontsize=12)
ax.set_ylabel('Score', fontweight='bold', fontsize=12)
ax.set_title('Key Classification Metrics Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos + width)
ax.set_xticklabels(key_metrics)
ax.set_ylim([0, 1.05])
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Confusion matrices
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
cm_lgb = confusion_matrix(y_test, y_pred_lgb)
cm_lgb_dist = confusion_matrix(y_test, y_pred_lgb_dist)

cms = [cm_xgb, cm_lgb, cm_lgb_dist]
titles = ['XGBoost', 'LightGBM', 'LightGBM Distributed']

for idx, (cm, title) in enumerate(zip(cms, titles)):
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx], cbar=False)
    
    axes[idx].set_title(f'{title} Confusion Matrix', fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

In [ ]:
# Model efficiency (accuracy vs inference time)
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

model_names = list(results.keys())
accuracies = [results[model]['Accuracy'] for model in model_names]
times = [results[model]['Inference Time (s)'] for model in model_names]

scatter = ax.scatter(times, accuracies, s=300, c=colors, alpha=0.6, edgecolors='black', linewidth=2)

for i, model in enumerate(model_names):
    ax.annotate(
        model,
        (times[i], accuracies[i]),
        xytext=(10, 10),
        textcoords='offset points',
        fontsize=10,
        fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.5', facecolor=colors[i], alpha=0.3),
    )

ax.set_xlabel('Inference Time (seconds)', fontweight='bold', fontsize=12)
ax.set_ylabel('Accuracy', fontweight='bold', fontsize=12)
ax.set_title('Model Efficiency: Accuracy vs Inference Time', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_ylim([min(accuracies) - 0.05, 1.05])

plt.tight_layout()
plt.show()

In [ ]:
# Ranking analysis
print("" + "=" * 80)
print("MODEL RANKING ANALYSIS")
print("=" * 80)

for metric in metrics + ['Inference Time (s)']:
    print(f"{metric} Rankings:")

    metric_values = [(model, results[model][metric]) for model in results.keys()]
    
    if metric == 'Inference Time (s)':
        metric_values.sort(key=lambda x: x[1])
        print("(Lower is Better)")

    else:
        metric_values.sort(key=lambda x: x[1], reverse=True)
        print("(Higher is Better)")

    for rank, (model, value) in enumerate(metric_values, 1):
        print(f"  {rank}. {model}: {value:.4f}")
              
print("" + "=" * 80)
print("OVERALL MODEL RANKING (Average Performance Across All Metrics)")
print("=" * 80)

overall_ranks = {model: [] for model in results.keys()}

for metric in metrics + ['Inference Time (s)']:
    metric_values = [(model, results[model][metric]) for model in results.keys()]

    if metric == 'Inference Time (s)':
        metric_values.sort(key=lambda x: x[1])

    else:
        metric_values.sort(key=lambda x: x[1], reverse=True)

    for rank, (model, _) in enumerate(metric_values, 1):
        overall_ranks[model].append(rank)
                                    
avg_ranks = {model: np.mean(ranks) for model, ranks in overall_ranks.items()}
sorted_models = sorted(avg_ranks.items(), key=lambda x: x[1])
                       
for rank, (model, avg_rank) in enumerate(sorted_models, 1):
    print(f"{rank}. {model}: Average Rank = {avg_rank:.2f}")

print("\n" + "=" * 60)
print("Evaluation Complete")
print("=" * 60)

### **SHAP Explainations of both Gradient Boosting Models**

In [ ]:
# SHAP explanation for XGBoost model
explainer_xgb = shap.TreeExplainer(xgb_clf) # setup SHAP tree explainer for the xgboost model
shap_values_xgb = explainer_xgb.shap_values(X_test.values) # compute SHAP values for the xgboost model using the testing data of the heart disease dataset

shap.summary_plot(shap_values_xgb, X_test, show=False) # summary plot the calculated SHAP values for the xgboost model

plt.title('SHAP Summary Plot for XGBoost')

plt.show() # output SHAP summary plot for the xgboost model

In [ ]:
# SHAP explanation for non-distributed LightGBM
explainer_lgb = shap.TreeExplainer(lgb_clf) # setup SHAP tree explainer for the non-distributed lightgbm model
shap_values_lgb = explainer_lgb.shap_values(X_test.values) # compute SHAP values for the non-distributed lightgbm model using the testing data of the heart disease dataset

shap.summary_plot(shap_values_lgb, X_test, show=False) # summary plot the calculated SHAP values for the non-distributed lightgbm model

plt.title('SHAP Summary Plot for LightGBM')

plt.show() # output SHAP summary plot for the non-distributed lightgbm model

In [ ]:
# SHAP explanation for LightGBM with Dask Distributed Learning
explainer_lgb_dist = shap.TreeExplainer(lgb_dist_clf) # setup SHAP tree explainer for the lightgbm model trained with dask distributed learning
shap_values_lgb_dist = explainer_lgb_dist.shap_values(X_test.values) # compute SHAP values for the lightgbm model trained with dask distributed learning using the testing data of the heart disease dataset

shap.summary_plot(shap_values_lgb_dist, X_test, show=False) # plot the calculated SHAP values for the lightgbm model trained with dask distributed learning

plt.title('SHAP Summary Plot for LightGBM with Dask')

plt.show() # output SHAP summary plot for the lightgbm model trained with dask distributed learning